[View in Colaboratory](https://colab.research.google.com/github/mlaskowski17/Reinforcement-Learning/blob/master/1.%20Markov%20Decision%20Processes/Google_Dopamine.ipynb)

## GOOGLE DOPAMINE



In [0]:
# @title Install necessary packages.

# dopamine library for RL
!pip install --upgrade --no-cache-dir dopamine-rl
# one of opamine dependencies
!pip install cmake
# Arcade Learning Environment
!pip install atari_py

In [0]:
# @title Necessary imports and globals.

# matrix math
import numpy as np
# load files
import os
# dopamine framework (DQN for baselines)
from dopamine.agents.dqn import dqn_agent
# high level agent-environment excecution engine
from dopamine.atari import run_experiment
# visualization + data downloading
from dopamine.colab import utils as colab_utils
# warnings
from absl import flags

#w here to store training logs
BASE_PATH = '/tmp/colab_dope_run'  # @param
# which arcade environment?
GAME = 'Asterix'  # @param

In [5]:
# @title Create a  new agent from scratch.

# tell the agent where to store log data
LOG_PATH = os.path.join(BASE_PATH, 'basic_agent', GAME)


# skeleton that we need for Google Dopamine 
# This is a way of defining structure, whatever our actions will be
class BasicAgent(object):
  """This agent randomly selects an action and sticks to it. It will change
  actions with probability switch_prob."""
  
  def __init(self, sess, num_actions, switch_prob=0.1):
    # tensorflow session
    self.sess = sess
    # how many possible actions can it take?
    self._num_actions = num_actions
    # probability of switching actions in the next timestep?
    self._switch_prob = switch_prob
    # initialize the action to take (randomly)
    self._last_action = np.random.randint(num_actions)
    # not debugging
    self.eval_mode = False
    
  # How select an action? 
  # we define our policy here (we choose ranodm action)
  def _choose_action(self):
    if np.random.random() <= self._switch_prob:
      self._last_action = np.random.randint(self._num_actions)
    return self._last_action
    
    
  #when it checkpoints during training, anything we should do?
  def bundle_and_checkpoint(self, unused_checkpoint_dir, unused_iteration):
    pass
    
  #loading from checkpoint
  def unbundle(self, unused_checkpoint_dir, unused_checkpoint_version,
               unused_data):
    pass
  
  #first action to take
  def begin_episode(self, unused_observation):
    return self._choose_action()
  
  #cleanup
  def end_episode(self, unused_reward):
    pass
  
  
  #we can update our policy here
  #using the reward and observation
  #dynamic programming, Q learning, monte carlo methods, etc.
  def step(self, reward, observation):
    return self._choose_action()
  
def create_basic_agent(sess, environment):
  """The Runner class will expect a function of this type to create an agent."""
  return BasicAgent(sess, num_actions=environment.action_space.n,
                     switch_prob=0.2)

# Create the runner class with this agent. We use very small numbers of steps
# to terminate quickly, as this is mostly meant for demonstrating how one can
# use the framework. We also explicitly terminate after 110 iterations (instead
# of the standard 200) to demonstrate the plotting of partial runs.
basic_runner = run_experiment.Runner(LOG_PATH,
                                      create_basic_agent,
                                      game_name=GAME,
                                      num_iterations=200,
                                      training_steps=10,
                                      evaluation_steps=10,
                                      max_steps_per_episode=100)

TypeError: ignored